In [1]:
# Import necessary modules

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
import sys
import math
from geopy.distance import distance
from matplotlib.colors import to_rgba
from collections import defaultdict

In [2]:
# Establish directory locations

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')

In [3]:
# Plotting preferences

# Global

start_date = datetime(2024, 5, 20) #YYYY, M, DD
end_date = datetime(2024, 5, 29)

min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

# Float colors
# For additional floats, add the name (as it appears in the 'Float_positions.csv') and color you'd like (as a hexcode)
float_colors = {
        '4903532':  '#B4184C',
        'navis102': '#F5A300',
        '1902637':  '#0000E0',
        'navis101': '#FBFF1F'}

# CHLA
skip_chla            = True
CHLA_color           = 'YlGnBu_r'
CHLA_plot_as_log     = True # Default True
CHLA_plot_lim_max    = 10   # Default 10
CHLA_plot_lim_min    = 0.1  # Default 0.1

# SST
skip_sst             = True
SST_color            = 'YlOrRd'
SST_plot_as_log      = False # Default False
SST_plot_lim_max     = 15    # Default 15
SST_plot_lim_min     = 5     # Default 5

# SSH
skip_sla             = True
SSH_color            = 'RdBu'
SSH_plot_as_log      = False # Default False
SSH_plot_lim_max     = 0.5   # Default  0.5
SSH_plot_lim_min     = -0.5  # Default -0.5

# PIC
skip_pic             = False
PIC_color            = 'YlGnBu_r'
PIC_plot_as_log      = True
PIC_plot_lim_max     = 0.1
PIC_plot_lim_min     = 0.0000001

In [4]:
# Function to generate points on a circle centered at (lat, lon) with a given radius
def generate_circle_points(lat, lon, radius_km, num_points=360):
    angles = np.linspace(0, 360, num_points)
    circle_points = []
    for angle in angles:
        point = distance(kilometers=radius_km).destination((lat, lon), angle)
        circle_points.append((point.latitude, point.longitude))
    
    return np.array(circle_points)

# Generates a list of dates between given dates 
def get_dates_between(start_date, end_date):
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    return dates

In [5]:
### Plotting most recent data from CMEMS
## List most recent files

# List to store matching file names
matching_files = []

# Loop through the files in the directory
for filename in os.listdir(NEODASS_dir):
    if filename.endswith('.nc') and f'-1d-' in filename:
        matching_files.append(filename)

# Dictionary to store files by their base names (excluding '_a_' or '_b_') and key variables
file_dict = {}

for file in matching_files:
    # Identify the base name by removing '_a' or '_b' if present
    base_name = file.replace('_a', '_').replace('_b', '_')
    if base_name not in file_dict:
        file_dict[base_name] = {}
    filepath = os.path.join(NEODASS_dir, file)
    ds = xr.open_dataset(filepath)
    variables = list(ds.data_vars.keys())
    print(variables)

    # Find the key variable excluding 'longitude' and 'latitude'
    key_variables = [var for var in variables if 'longitude' not in var.lower() and 'latitude' not in var.lower()]

    if key_variables:
        time_str = ds['time'].dt.strftime("%Y-%m-%d").item()
        for key_variable in key_variables:
            file_dict[base_name][file] = {
                'variable': key_variable,
                'date': datetime.strptime(time_str, "%Y-%m-%d")
                }       
    else:
        print(f"No suitable key variable found in file: {file}")

    ds.close()

# Group files by variable category and day
var_dict = {'chl': {}, 'temp': {}, 'sla': {}, 'adt': {}, 'bbp': {}, 'pic': {}}

for base_name, files in file_dict.items():
    for file, data_info in files.items():
        data_var = data_info['variable']
        date = data_info['date']
        var_name_lower = data_var.lower()
        print(data_var)
        if 'chl' in var_name_lower:
            if date not in var_dict['chl']:
                var_dict['chl'][date] = []
            var_dict['chl'][date].append((file, data_var))
        elif 'temp' in var_name_lower:
            if date not in var_dict['temp']:
                var_dict['temp'][date] = []
            var_dict['temp'][date].append((file, data_var))
        elif 'sla' in var_name_lower:
            if date not in var_dict['sla']:
                var_dict['sla'][date] = []
            var_dict['sla'][date].append((file, data_var))
        elif 'adt' in var_name_lower:
            if date not in var_dict['adt']:
                var_dict['adt'][date] = []
            var_dict['adt'][date].append((file, data_var))
        elif 'bbp' in var_name_lower:
            if date not in var_dict['bbp']:
                var_dict['bbp'][date] = []
            var_dict['bbp'][date].append((file, data_var))
        elif 'pic' in var_name_lower:
            if date not in var_dict['pic']:
                var_dict['pic'][date] = []
            var_dict['pic'][date].append((file, data_var))

# Print the results
for var_category, var_data in var_dict.items():
    print(f"{var_category.capitalize()} files:")
    for date, files in var_data.items():
        print(f"  Date: {date}")
        for file, variable in files:
            print(f"    File: {file}, Variable: {variable}")

# We load shape files for bathymetry lines
#first_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_J_1000.shp'
#second_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_I_2000.shp'

first_line_path  = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_J_1000.shp'
second_line_path = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_I_2000.shp'

gdf_1000 = gpd.read_file(first_line_path)
gdf_2000 = gpd.read_file(second_line_path)

['sla']
['sla']
['sla']
['sla']
['sla']
['sla']
['sla']
['sla']
['sla']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'CHL_OC4ME']
['latitude_longitude', 'sea_surface_temperature']
['latitude_longitude', 'sea_surface_temperature']
['latitude_longitude', 'sea_surface_temperature']
['latitude_longitude', 'sea_surface_temperature']
['latitude_longitude', 'sea_surface_temperature']
['latitude_longitude', 'sea_su

In [63]:
# Fixed features

# Circle around deploy point
circle_points = generate_circle_points(60, -24, 444)
circle_lats, circle_lons = circle_points[:, 0], circle_points[:, 1]

# Load float positions
float_position_path = os.path.join(floats_dir, 'Float_positions_2024_05_25.csv')
full_position = pd.read_csv(float_position_path)

# Date strings for plotting
dates_to_plot = get_dates_between(start_date, end_date)

for var_category, files in tqdm(var_dict.items(), desc = 'Progress Plotting'):
    
    if skip_chla and var_category == 'chl':
        continue
    if skip_sst and var_category == 'temp':
        continue
    if skip_sla and var_category == 'sla':
        continue
    if skip_pic and var_category == 'pic':
        continue

    for date, files_per_day in files.items():  # Iterate over each day

        if date.strftime("%Y-%m-%d") not in dates_to_plot:
            continue

        # Hook for catching dates to plot here?
          
        daily_data = []  # List to store data variables

        # If multiple files for the day, compute spatial mean        
        if len(files_per_day) > 1:

            for file, data_var in files_per_day:
                # Extract the variable and append to list
                cur_data_var = xr.open_dataset(os.path.join(NEODASS_dir, f'{file}'))[data_var]
                daily_data.append(cur_data_var)
            
            # Align all datasets spatially
            aligned_data = xr.align(*daily_data, join='override') # The override option here works as data is from same day.
            
            # Combine aligned data into a single dataset
            combined_data = xr.concat(aligned_data, dim='time').mean(dim='time', skipna=True)

            #print(f"Averaged data for {var_category} on {date}:")
            #print(combined_data)

        # If single file for the day, extract value
        else:

            for file, data_var in files_per_day:

                print(f"Working on '{file}'")

                cur_data_var = xr.open_dataset(os.path.join(NEODASS_dir, f'{file}'))[data_var]

                combined_data = cur_data_var.isel(time=0)

        date_of_plot = date.strftime("%Y-%m-%d")  # Format the date for plot title
        
        #print(*daily_data, sep='\n')
        #print(combined_data, sep='\n')

        if var_category == 'chl':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Chlorophyll (' + xx_plot_units + ')'
            xx_plot_min = CHLA_plot_lim_min
            xx_plot_max = CHLA_plot_lim_max
            color = CHLA_color
            log_scaling = CHLA_plot_as_log
            xx_output_dir_name = 'Chla'

        elif var_category == 'temp':
            xx_plot_units = cur_data_var.attrs.get('units')
            if xx_plot_units == 'kelvin':
                # Convert Kelvin to Celsius
                combined_data = combined_data - 273.15
                degree_sign = u'\N{DEGREE SIGN}'
                xx_plot_cbar_label = 'Sea Surface Temperature (' + degree_sign + 'C)'
                xx_plot_min = SST_plot_lim_min
                xx_plot_max = SST_plot_lim_max
                color = SST_color
                log_scaling = SST_plot_as_log
                xx_output_dir_name = 'SST'
                
        elif var_category == 'sla':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Sea level anomaly (' + xx_plot_units + ')'
            xx_plot_min = SSH_plot_lim_min
            xx_plot_max = SSH_plot_lim_max
            color = SSH_color
            log_scaling = SSH_plot_as_log
            xx_output_dir_name = 'SLA'

        elif var_category == 'adt':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Sea surface velocity (' + xx_plot_units + 's$^{-1}$)'
            xx_plot_min = SSH_plot_lim_min
            xx_plot_max = SSH_plot_lim_max
            color = SSH_color
            log_scaling = SSH_plot_as_log
            xx_output_dir_name = 'SSC'

        elif var_category == 'bbp':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Particulate backscatter (' + xx_plot_units + ')'
            xx_plot_min = BBP_plot_lim_min
            xx_plot_max = BBP_plot_lim_max
            color = BBP_color
            log_scaling = BBP_plot_as_log
            xx_output_dir_name = 'BBP'

        elif var_category == 'pic':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'PIC (' + xx_plot_units + ')'
            xx_plot_min = PIC_plot_lim_min
            xx_plot_max = PIC_plot_lim_max
            color = PIC_color
            log_scaling = PIC_plot_as_log
            xx_output_dir_name = 'PIC'

        # Dependant settings based on each variable type
        else:
            print(f"No settings set for '{var_category}', skipping!")
            continue
        
        norm = colors.LogNorm(vmin=xx_plot_min, vmax=xx_plot_max) if log_scaling else colors.Normalize(vmin=xx_plot_min, vmax=xx_plot_max) # Setting color scale based on Log options.
        print(combined_data.max())
        print(combined_data.min())
        # Plotting

        if var_category == 'adt':
    
                fig = plt.figure(figsize=(15, 15))
                ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
                ax.set_extent([-30, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
    
                # Extract the lon and lat from the dataset only once
                x = combined_data['longitude'].data
                y = combined_data['latitude'].data
    
                #From the U and V vector compute the speed, we use it as our colour map
                u = combined_data['ugos'].isel(time=day_offset).data
                v = combined_data['vgos'].isel(time=day_offset).data
                speed = np.sqrt(u**2 + v**2)
    
                #Plot the current vectors field and the coastline
                im = ax.quiver(x, y, u, v, speed, angles='xy', scale_units='xy', cmap='viridis', transform=ccrs.PlateCarree(), regrid_shape=50, width = 0.002)
    
                cbar = plt.colorbar(im, ax=ax, label=xx_plot_cbar_label)
        
        else:

            combined_data = np.clip(combined_data, xx_plot_min, xx_plot_max)
            fig = plt.figure(figsize=(20, 10))
            ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
            ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
            im = ax.pcolormesh(combined_data['longitude'], combined_data['latitude'], 
                               combined_data, 
                               cmap=color, 
                               norm=norm, 
                               transform=ccrs.PlateCarree())
            
            cbar = plt.colorbar(im, ax=ax, label=xx_plot_cbar_label)

        # Float plotting section
        # Get the unique float names
        unique_floats = full_position['float'].unique()

        # Store legend handles and labels
        legend_handles = []
        
        # Plot each float's positions
        for float_name in unique_floats:
            float_data = full_position[full_position['float'] == float_name].sort_values(by='JULD', ascending=False)  # Sort in descending order

            # Get the colors and opacities for the points
            rgba_color = to_rgba(float_colors.get(float_name, '#000000'))  # Default to black if not found
            num_positions = len(float_data)
            alphas = np.linspace(1, 0.1, min(num_positions, 9))  # Reverse the opacity gradient

        # Plot the positions and lines, with the first 9 points have a decreasing alpha
            for i, (index, row) in enumerate(float_data.iterrows()):
                alpha = alphas[i] if i < 9 else 0.1
                rgba_color_with_alpha = (*rgba_color[:3], alpha)
                marker_edge_color     = (*rgba_color[:3], alpha)  # Marker edge color follows opacity gradient
                ax.scatter(row['LONGITUDE'], row['LATITUDE'], color=rgba_color_with_alpha, edgecolor=marker_edge_color, transform=ccrs.PlateCarree(), zorder=3)
    
                if i > 0:
                    prev_row = float_data.iloc[i - 1]
                    ax.plot([prev_row['LONGITUDE'], row['LONGITUDE']],
                            [prev_row['LATITUDE'], row['LATITUDE']],
                            color=rgba_color_with_alpha, transform=ccrs.PlateCarree(), zorder=2)
                    
                # Create a proxy artist for each float
                proxy = plt.Line2D([0], [0], linestyle='-', marker='o', color=float_colors.get(float_name, '#000000'), label=float_name)
                legend_handles.append(proxy)
    
        # Deploy point position marker
        sc3 = ax.scatter(-24,60,transform=ccrs.PlateCarree(), edgecolors='black', facecolors='none', marker='s')
        ax.plot(circle_lons, circle_lats, transform=ccrs.PlateCarree(), color='black')
        
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        # Add coastlines
        ax.add_feature(cfeature.COASTLINE)

        # Add lon/lat grid and adjust labels
        gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels = False
        gl.right_labels = False

        # Plot title
        plot_title = f'NEODASS_{xx_output_dir_name} {date_of_plot}'
        ax.set_title(plot_title, fontsize=24)

        # Add legend in
        deploy_point_proxy = plt.Line2D([0], [0], linestyle='none', marker='s', color='black', markersize=10, markerfacecolor='none', label='Deploy Point')
        legend_handles.append(deploy_point_proxy)
        ax.legend(handles=legend_handles, loc='lower left')

        # Save output
        save_dir = os.path.join(output_dir, f'{xx_output_dir_name}{"_log" if log_scaling else ""}_TEST')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        basename_suffix = base_name[:10]
        filename = f'{xx_output_dir_name}{"_log" if log_scaling else ""}_{date_of_plot}_NEODASS.png'
        plt.savefig(os.path.join(save_dir, filename))
        
        plt.clf()
        plt.close()
        

Working on 'viirs_suomi-chl_oc5_pic-1d-20240520_20240520-nrt.nc'
<xarray.DataArray 'pic' ()> Size: 4B
array(0.008686, dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2024-05-20T23:54:01.140000
<xarray.DataArray 'pic' ()> Size: 4B
array(9.998679e-06, dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2024-05-20T23:54:01.140000
Working on 'viirs_suomi-chl_oc5_pic-1d-20240521_20240521-nrt.nc'
<xarray.DataArray 'pic' ()> Size: 4B
array(0.004482, dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2024-05-21T23:36:00.670000128
<xarray.DataArray 'pic' ()> Size: 4B
array(9.998679e-06, dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2024-05-21T23:36:00.670000128
Working on 'viirs_suomi-chl_oc5_pic-1d-20240522_20240522-nrt.nc'
<xarray.DataArray 'pic' ()> Size: 4B
array(0.007916, dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2024-05-22T16:18:00.400000
<xarray.DataArray 'pic' ()> Size: 4B
array(9.998679e-06, dtype=float32)
Coordinates:
    time   